# XoT Tutorial

In [1]:
%load_ext autoreload
%autoreload 2

## Prepare your OpenAI GPT Model

Import OpenAI model with langchain.

In [2]:
import os
from dotenv import load_dotenv

import langchain
from langchain_openai import AzureChatOpenAI, ChatOpenAI

from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(
    SQLiteCache(database_path=".langchain_cache.db")
)  # setup cache for the LLM

langchain.debug = False  # set verbose mode to True to show more execution details

load_dotenv()

# Use Azure's models
langchain_llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("MY_AZURE_ENDPOINT"),
    api_key=os.getenv("MY_AZURE_API_KEY"),
    api_version=os.getenv("MY_AZURE_API_VERSION"),
    azure_deployment=os.getenv("MY_AZURE_DEPLOYMENT_NAME"),
    verbose=True,
)

# Or use OpenAI's model
langchain_llm = ChatOpenAI(
    base_url=os.getenv("MY_OPENAI_API_BASE"),
    api_key=os.getenv("MY_OPENAI_API_KEY"),
    model="gpt-4o",
    verbose=True,
)

In [ ]:
# a debug line to show you have the correct model setup
langchain_llm.invoke("What is LLM?")

## Direct Use of LLM

* Practice the use of LLM with langchain.
* Intent classifier: does the user want to have its tablet repaired?

In [ ]:
from utils.intent_classifier import intent_classifier_prompt, intent_classifier_parser

intent_classifier_chain = (
    intent_classifier_prompt | langchain_llm | intent_classifier_parser
)

In [ ]:
prompt = intent_classifier_prompt.invoke({"user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."})
output = langchain_llm.invoke(prompt)
intent_classifier_parser.invoke(output)

In [ ]:
intent_classifier_chain.invoke(
    {"user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."}
)

## Chain of Thoughts Prompting

* Practice the use of CoT with langchain.
* Use warranty checker as an example.

In [ ]:
from utils.warranty_module import warranty_parser, warranty_checker_prompt


warranty_checker_chain = warranty_checker_prompt | langchain_llm

In [ ]:
warranty_checker_prompt.invoke({"user_input": "I want to check my warranty status."}).to_messages()

In [ ]:
output = warranty_checker_chain.invoke({"user_input": "I bought this item on May 1st, 2023."})
warranty_parser.invoke(output)

## Tree of thoughts

* We use tree of thoughts to practice repair plan suggestion. Available plans are repair, replace and trade-in options. 
* 

In [ ]:
from utils.repair_strategy import repair_strategy_prompt, repair_parser
from utils.replace_strategy import replacement_strategy_prompt, replacement_parser
from utils.tradein_strategy import tradein_strategy_prompt, tradein_parser

repair_strategy_chain = repair_strategy_prompt | langchain_llm
replacement_strategy_chain = replacement_strategy_prompt | langchain_llm
tradein_strategy_chain = tradein_strategy_prompt | langchain_llm

In [ ]:
repair_strategy_chain.invoke(
    {"user_input": "I want to repair my device. The screen and the battery are broken."}
)
repair_parser.invoke(
    repair_strategy_chain.invoke(
        {
            "user_input": "I want to repair my device. The screen and the battery are broken."
        }
    )
)

In [ ]:
tradein_strategy_chain.invoke(
    {"user_input": "I want to trade in my device. The keyboard is not working"}
)
tradein_parser.invoke(
    tradein_strategy_chain.invoke(
        {"user_input": "I want to trade in my device. The keyboard is not working"}
    )
)

In [ ]:
replacement_strategy_chain.invoke({"user_input": "The screen is broken, I want to replace it.", "additional_info": "No warranty"})
replacement_parser.invoke(replacement_strategy_chain.invoke({"user_input": "The screen is broken, I want to replace it.", "additional_info": "No warranty"}))

In [ ]:
from utils.plan_selection import plan_selection_prompt, plan_parser

plan_evaluater_chain = plan_selection_prompt | langchain_llm

In [ ]:
plan_evaluater_chain.invoke(
    {
        "replace_plan": "I'm sorry, but you cannot get a replacement. Only products with warranty can be replaced.",
        "repair_plan": "The total cost to fix the product is 150 USD.",
        "tradein_plan": "You can get a 100 USD coupon if you want to trade in your old device for a new one.",
        "user_preference": "None",
        "selected_plan": "repair",
        "ai_analysis": "Let's thing step by step. The cost for a new device is 249 USD, the user can get a coupon so he will spend 149 USD for the new device. The cost for fixing the old device is 150 USD. Therefore, it is better to buy a new device as the cost is almost the same.",
        "ai_response": "Maybe you should consider buying a new device instead of fixing the old one. The cost for fixing the old device is 150 USD, and the cost for a new device is 249 USD. You can get a 100 USD coupon if you want to trade in your old device for a new one. The total cost to fix the product is 150 USD.",
    }
)
plan_parser.invoke(
    plan_evaluater_chain.invoke(
        {
            "replace_plan": "Your device can be replaced.",
            "repair_plan": "The total cost to fix the product is 150 USD.",
            "tradein_plan": "You can get a 100 USD coupon if you want to trade in your old device for a new one.",
            "user_preference": "None",
        }
    )
)

In [ ]:
from langchain_core.runnables import RunnableParallel

# The tot structure
repair_strategy_chain = repair_strategy_prompt | langchain_llm | repair_parser
replacement_strategy_chain = (
    replacement_strategy_prompt | langchain_llm | replacement_parser
)
tradein_strategy_chain = tradein_strategy_prompt | langchain_llm | tradein_parser

tot_generater = RunnableParallel(repair_plan = repair_strategy_chain, replace_plan = replacement_strategy_chain, tradein_plan = tradein_strategy_chain)

plan_output = tot_generater.invoke({
    "user_input": "The screen is broken, I want to replace it.",
    "additional_info": "No warranty"
})

tot_scorer = plan_selection_prompt | langchain_llm | plan_parser

tot_scorer.invoke({
    "replace_plan": plan_output["replace_plan"]["Utterance"],
    "repair_plan": plan_output["repair_plan"]["Utterance"],
    "tradein_plan": plan_output["tradein_plan"]["Utterance"],
    "user_preference": "None",
})

## GoT structure

* Break down the repair plan
* The information needed are warranty status, broken places, and the user's preference on whether to get a new device or not.


In [ ]:
# upon receiving user input, all nodes will update its internal state
from utils.techissue_analyser import tech_issue_analyser_prompt, tech_issue_analyser_parser

tech_issue_analyser_chain = tech_issue_analyser_prompt | langchain_llm | tech_issue_analyser_parser

In [ ]:
tech_issue_analyser_chain.invoke(
    {
        "chat_history": "I bought this item on May 1st, 2023. \n The screen is broken, I want to replace it. \n It does not charge at the moment",
        "user_input": "Please help."
    }
)

In [ ]:
warranty_checker_fullchain = warranty_checker_prompt | langchain_llm | warranty_parser
warranty_checker_fullchain.invoke({"user_input": "I bought this item on May 1st, 2023."})

In [ ]:
# A simple GoT Prompter: decide what question to ask based on the required information.
from utils.question_generator import question_generator_prompt, question_generator_parser # The parser

question_generator_chain = question_generator_prompt | langchain_llm | question_generator_parser

question_generator_chain.invoke(
    {
        "required_information": "preference (how much repair fee can afford?)"
    }
)

# A simple GoT Controller: ask twice for the required information, and then generate a plan
# See the chatbot


In [ ]:
from chatbot import Chatbot

customer_chatbot = Chatbot()

In [ ]:
customer_chatbot.interact("hello")

In [ ]:
customer_chatbot.interact("I want to repair my tablet")

In [ ]:
customer_chatbot.interact("Yeah, the screen is always black and there is no audio")